In [ ]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns 

from datetime import datetime, timedelta

from scipy.stats import mannwhitneyu, chi2_contingency, ttest_ind
from statsmodels.stats.proportion import proportion_confint

from sklearn.cluster import KMeans

# Введение

In [ ]:
delivery_data =pd.read_csv("restaurant-orders.csv")

In [ ]:
delivery_data.columns

In [ ]:
delivery_data.isna().sum()

In [ ]:
delivery_data.head()

# Предобработка данных

In [ ]:
delivery_data = delivery_data.rename(columns={
    "Time customer placed order":"time_ordered",
    "Time order placed at restaurant":"time_placed",
    "Time driver arrived at restaurant":"time_arrived",
    "Delivery time":"time_delivered",
    "Driver ID":"driver_id",
    "Driver Name":"driver_name",
    "Restaurant ID":"restaurant_id",
    "Customer ID":"customer_id",
    "Delivery Area":"delivery_area",
    "Sub Total":"sub_total",
    "Delivery fee":"delivery_fee",
    "Service fee":"service_fee",
    "Discount":"discount",
    "Tip":"tip",
    "Refunded amount":"refund"
})

In [ ]:
#ML модель основывается на данных для пользователя, колонки водителей и ресторанов излишни
delivery_data = delivery_data.drop(["driver_name","restaurant_id","driver_id"],axis=1)

#Время готовки позиции в ресторане и время приезда курьера в него можно опустить (кроме того данные содержат пропуски)
delivery_data = delivery_data.drop(["time_placed","time_arrived"], axis = 1)


In [ ]:
def get_normal_price(refund):
    """получить цену без $"""
    refund = refund[1:]
    return float(refund)

In [ ]:
#Преобразовать цены
delivery_data["refund"] = delivery_data.refund.astype(str).apply(lambda x: get_normal_price(x))
delivery_data["tip"] = delivery_data.tip.astype(str).apply(lambda x: get_normal_price(x))
delivery_data["discount"] = delivery_data.discount.astype(str).apply(lambda x: get_normal_price(x))
delivery_data["service_fee"] = delivery_data.service_fee.astype(str).apply(lambda x: get_normal_price(x))
delivery_data["delivery_fee"] = delivery_data.delivery_fee.astype(str).apply(lambda x: get_normal_price(x))

#есть формат цены 1,000.14
delivery_data["sub_total"] = delivery_data.sub_total.str.replace(",","")
delivery_data["sub_total"] = delivery_data.sub_total.astype(str).apply(lambda x: get_normal_price(x))

In [ ]:
delivery_data.head()

# Feature engineering (Базовый)

## Калькуляция времени доставки

In [ ]:
#преобразовываем строковое время в удобный формат времени
time_ordered = delivery_data.time_ordered.apply(lambda x: datetime.strptime(x,"%H:%M:%S"))
time_delivered = delivery_data.time_delivered.apply(lambda x: datetime.strptime(x,"%H:%M:%S"))

time_delta_df = pd.DataFrame({"time_ordered":time_ordered,
                              "time_delivered":time_delivered})

In [ ]:
def get_delivery_minutes(t_ordered, t_delivered):
    # Если время доставки меньше времени заказа, значит, доставка на следующий день
    if t_delivered < t_ordered:
        t_delivered += timedelta(days=1)  # Добавляем 1 день
    
    delta = t_delivered - t_ordered
    return round(delta.total_seconds() / 60, 2)  # Округляем до минут

def get_delivery_time(row):
    dt_ordered = row["time_ordered"]
    dt_delivered = row["time_delivered"]
    return get_delivery_minutes(dt_ordered,dt_delivered)

In [ ]:
#общее время доставки
delivery_time = time_delta_df.apply(get_delivery_time, axis=1)

delivery_data["delivery_time"] = delivery_time

In [ ]:
delivery_data.head()

## Вычисляем день недели
Распределим даты по дням (понедельник, вторник, среда и т.д)

In [ ]:
days = pd.to_datetime(delivery_data.Date)

delivery_data["Date"] = days.dt.day_name()

In [ ]:
delivery_data.head()

# EDA

## Анализ распределений

### Распределение классов

Неравномерное распределение классов, необходимо уравновешивание классов по формуле: $$n_{samples}/ (n_{classes} * np.bincount(y))$$

In [ ]:
refunds = delivery_data[delivery_data.refund>0].customer_id.count()
non_refunds = delivery_data[delivery_data.refund==0].customer_id.count()

print(f"Количество возратов: {refunds}")
print(f"Количество оформленных заказов: {non_refunds}")
print(f"В процентном соотношении: {refunds / (non_refunds+refunds) *100:.2f}%")

#### Распределение классов по дням недели

In [ ]:
date_table = (
    delivery_data
    .assign(refund=lambda x: x['refund'] > 0)  # Создаем временный столбец без копирования
    .pivot_table(
        index="refund",
        values="customer_id",
        columns="Date",
        aggfunc="count",
        fill_value=0
    )
    .reset_index()
)
date_table

In [ ]:
date_table.drop(columns='refund').sum(axis=1)

### Время доставки

In [ ]:
sns.kdeplot(delivery_data,x="delivery_time")

### Цена доставки

In [ ]:
sns.kdeplot(delivery_data,x="sub_total")

#### Аномалия

На графике можно заметить, что цена некотых заказов нулевая

In [ ]:
print(f"Количество таких наблюдений: {delivery_data[delivery_data.sub_total==0].customer_id.count()}")

In [ ]:
delivery_data[delivery_data.sub_total==0].head()

Возможные причины такого поведения цены включают ошибки транзакций, ошибки со стороны базы данных. Возможно это даже некоторая мошенническая махинация или системная уязвимость, в любом случае наблюдения любопытны и будут устранены

In [ ]:
#устранение аномалий
delivery_data = delivery_data[delivery_data.sub_total!=0]
sns.kdeplot(delivery_data,x="sub_total")

### Побочные ценовые компоненты

In [ ]:
sns.kdeplot(delivery_data,x="delivery_fee")

In [ ]:
sns.kdeplot(delivery_data,x="service_fee")

In [ ]:
sns.kdeplot(delivery_data,x="discount")

In [ ]:
sns.kdeplot(delivery_data,x="tip")

### Дни недели

In [ ]:
sns.countplot(delivery_data,x="Date")

# Устранение выбросов

## Выбросы веремени доставки

In [ ]:
sns.kdeplot(delivery_data,x="delivery_time")

In [ ]:
normalized_delivery = 1 / delivery_data.delivery_time
sns.kdeplot(normalized_delivery)

In [ ]:
print(f"99-й персептиль: {normalized_delivery.quantile(0.99):.6f}")

#сметим все значения к 99-ому пернсептилю
normalized_delivery = normalized_delivery.clip(upper = normalized_delivery.quantile(0.99))

In [ ]:
sns.kdeplot(normalized_delivery)

In [ ]:
delivery_data["delivery_time"] = normalized_delivery

bin =[0, 0.0016, 0.002, 1]  #подобрано через опущенную визуализацию
labels = ["extra_long", "long", "normal"]
delivery_data["delivery"] = pd.cut(delivery_data["delivery_time"],bins=bin,labels=labels)

In [ ]:
sns.kdeplot(delivery_data, x="delivery_time",hue="delivery")

In [ ]:
sns.kdeplot(delivery_data[(delivery_data.delivery =="extra_long") | (delivery_data.delivery == "long")], x= "delivery_time", hue="delivery")

In [ ]:
sns.kdeplot(delivery_data[(delivery_data.delivery =="normal")], x= "delivery_time")

## Выбросы цены

Для цены товара используем обратную конвертацию ("сжимаем большие цены к началу, отдавая приоритет меньшим значениям")

In [ ]:
delivery_data["sub_total"] = 1 / delivery_data.sub_total

In [ ]:
print(f"99-й персенптиль обратной стоимости товара: {delivery_data.sub_total.quantile(0.99):.2f}")\

threshold = delivery_data.sub_total.quantile(0.99)
# delivery_data = delivery_data[delivery_data.sub_total <=threshold]

delivery_data["sub_total"] = delivery_data["sub_total"].clip(upper=threshold)

In [ ]:
sns.kdeplot(delivery_data,x="sub_total")

## Выбросы скидок и чаевых

In [ ]:
#заменим ценовой discount на экивалент % от стоимости 
delivery_data["discount"] = (delivery_data.discount / (1/delivery_data.sub_total)) *100

#тоже самое с tip
delivery_data["tip"] = (delivery_data.tip / (1/delivery_data.sub_total) )*100

In [ ]:
sns.kdeplot(delivery_data,x="discount")

In [ ]:
print(f"Максимальная скидка из данных: {delivery_data.discount.max():.2f}")

In [ ]:
bins = [0,5,10,20,50,92]
labels = ['0-5%', '5-10%', '10-20%', '20-50%', '50-92%']
delivery_data["discount"] = pd.cut(delivery_data.discount,bins=bins,labels=labels)

In [ ]:
sns.countplot(delivery_data,x="discount")

In [ ]:
sns.kdeplot(delivery_data,x="tip")

In [ ]:
print(f"Максимальные чаевые из данных: {delivery_data.tip.max():.2f}")

In [ ]:
bins = [0,5,10,20,50,90]
labels = ['0-5%', '5-10%', '10-20%', '20-50%', '50-90%']
delivery_data["tip"] = pd.cut(delivery_data.tip,bins=bins,labels=labels)

In [ ]:
sns.countplot(delivery_data,x="tip")

# Замена классифицируемой переменной на фактор
Исследуется факт возврата, а не вероятное количество возращаемой суммы.

In [ ]:
delivery_data["refund"] = delivery_data.refund >0

# updated EDA Анализ взаимодействий переменных

## Скидки

In [ ]:
sns.countplot(delivery_data,x="discount",hue="refund")

### Рассмотрим поближе экстремальные случаи

In [ ]:
sns.countplot(delivery_data[(delivery_data.discount=="20-50%") | (delivery_data.discount=="50-92%")],x="discount",hue="refund")

Анализ показывает, что возвраты происходят чаще, если скидка маленькая и реже, если скидка большая

In [ ]:
sns.lineplot(delivery_data,x= "discount",y="sub_total")

In [ ]:
sns.lineplot(delivery_data,y="delivery_time",x="discount")

## Сторонние фаторы цены

In [ ]:
sns.lineplot(delivery_data,x="delivery_fee", y="sub_total")

In [ ]:
sns.lineplot(delivery_data,x="service_fee",y="sub_total")

### Корреляция с возвратом

In [ ]:
sns.boxplot(delivery_data, x="service_fee", hue="refund")

In [ ]:
sns.pointplot(delivery_data, y="service_fee", x="refund")

In [ ]:
sns.boxplot(delivery_data, x="delivery_fee", hue="refund")

In [ ]:
sns.pointplot(delivery_data, y="delivery_fee", x="refund")

Даже если различия статистически значимы, смещение медиан в обоих случаях настоллько незначительное, что практически не дает нам данных.
Кроме того, переменная стоимости сервиса коррелирует со стоимостью.
Было решено объединить эти параметры для улучшения модели

## Время доставки

In [ ]:
sns.boxplot(delivery_data,x = "delivery_time", hue="refund")

In [ ]:
sns.pointplot(delivery_data,x="refund",y="delivery_time")

In [ ]:
sns.boxplot(delivery_data, x="ASAP",y="delivery_time", hue="refund")

In [ ]:
sns.pointplot(delivery_data,x="refund",y="delivery_time", hue = "ASAP")

## Чаевые

In [ ]:
sns.countplot(delivery_data,x="tip",hue="refund")

## Дни недели

In [ ]:
sns.countplot(delivery_data,x="Date",hue="refund")

# Feature engineering

## Объединяем цены

In [ ]:
#объединяем цены
delivery_data["sub_total"] = (1 / 
                              ((1/ delivery_data.sub_total) + delivery_data.delivery_fee + delivery_data.service_fee))

#старые переменные уже не нужны
delivery_data = delivery_data.drop(["delivery_fee","service_fee"],axis=1)

In [ ]:
sns.kdeplot(delivery_data, x="sub_total")

## Распределение по времени заказа
Распределим время как утро, день, вечер, ночь

In [ ]:
# ночь (00:00:00 - 04:59:59)
# утро (05:00:00 - 09:59:59)
# день (10:00:00 - 16:59:59)
# вечер (17:00:00 - 21:59:59)
# ночь (22:00:00 - 23:59:59)

labels = ["ночь", "утро", "день", "вечер", "ночь"]

# Преобразуем время в секунды
time_delivered_seconds =(
    time_delivered.dt.hour * 3600 +
    time_delivered.dt.minute *60 +
    time_delivered.dt.second
)

time_ordered_seconds =(
    time_ordered.dt.hour * 3600 +
    time_ordered.dt.minute *60 +
    time_ordered.dt.second
)

delivery_data["time_delivered"] = pd.cut(time_delivered_seconds,
        bins =[0,5*3600, 10*3600, 17*3600, 22*3600, 24*3600], 
        labels=labels, ordered = False)

delivery_data["time_ordered"] = pd.cut(time_ordered_seconds,
        bins =[0,5*3600, 10*3600, 17*3600, 22*3600, 24*3600], 
        labels=labels, ordered = False)

In [ ]:
delivery_data.head()

# Final EDA

## Зависимость дня недели и возврата

In [ ]:
sns.barplot(delivery_data,hue="Date", y="refund")

In [ ]:
date_refund_table = pd.crosstab(delivery_data.Date, delivery_data.refund)

In [ ]:
print(f"Уровень p-value Хи-квадрат теста: {chi2_contingency(date_refund_table).pvalue:.2f}")

## Промежуток времени оформления заказа и возврат

### Время заказа позиции

In [ ]:
sns.countplot(delivery_data,x="time_ordered")

In [ ]:
sns.barplot(delivery_data,y="refund", hue="time_ordered")

In [ ]:
pd.crosstab(delivery_data.time_ordered,delivery_data.refund)

In [ ]:
print(f"Уровень p-value Хи-квадрат теста: {chi2_contingency(pd.crosstab(delivery_data.time_ordered,delivery_data.refund)).pvalue:.2f}")

### Время приезда заказа клиенту

In [ ]:
sns.countplot(delivery_data,x="time_delivered")

In [ ]:
sns.barplot(delivery_data,y="refund",hue="time_delivered")

In [ ]:
pd.crosstab(delivery_data.time_delivered, delivery_data.refund)

In [ ]:
print(f"Уровень p-value Хи-квадрат теста: {chi2_contingency(pd.crosstab(delivery_data.time_delivered, delivery_data.refund)).pvalue:.2f}")

## Место проведения доставок

In [ ]:
sns.barplot(delivery_data,y="refund",hue="delivery_area")

In [ ]:
pd.crosstab(delivery_data.delivery_area, delivery_data.refund)

In [ ]:
print(f"Уровень p-value Хи-квадрат теста: {chi2_contingency(pd.crosstab(delivery_data.delivery_area, delivery_data.refund)).pvalue:.2f}")

## As Soon As Possible показатель

In [ ]:
sns.pointplot(delivery_data,y="refund",x="ASAP")

In [ ]:
cross_tab = pd.crosstab(delivery_data.ASAP,delivery_data.refund)
cross_tab

In [ ]:
print(f"Уровень p-value Хи-квадрат теста: {chi2_contingency(cross_tab).pvalue:.2f}")

сила связи по формуле Cramers V не подходит из-за дисбаланса классов. Посмотрим средние отношения и доверительные интервалы по ним

In [ ]:
proportions = delivery_data.groupby("ASAP")["refund"].mean()
print(f"Средний процент возврата в группе ASAP '{proportions.index[0]}': {proportions.iloc[0]:.3f}")

print(f"Средний процент возврата в группе ASAP '{proportions.index[1]}': {proportions.iloc[1]:.3f}")

print(f"Относительный риск между первой и 2-й группами: '{proportions.index[0]}': {proportions.iloc[0] / proportions.iloc[1]:.3f} ({((proportions.iloc[0] / proportions.iloc[1])-1)*100:.3f}%)")

Практическая значимость объясняется тем, что при факторе ASAP "No" вероятность возврата на 13% больше, чем с ASAP "Yes".
Вероятно причиной является взаимодействие факторов времени и требует дополнительного анализа

#### Взаимодействие с временем доставки

In [ ]:
sns.boxplot(delivery_data, y="delivery_time",x="ASAP",hue="refund")

In [ ]:
time_table = 1 / delivery_data.groupby(["refund","ASAP"]).delivery_time.mean()
print(f"Разница среднего времени в группе ASAP NO: {time_table.iloc[2] - time_table.iloc[0]:.2f}")
print(f"Разница среднего времени в группе ASAP YES: {time_table.iloc[3] - time_table.iloc[1]:.2f}")
time_table

Хоть разница и статистически значима, практическая ценность взаимодействия фактора ASAP и время доставки сомнительно

## Цена товара

In [ ]:
sns.boxplot(delivery_data, y="sub_total", hue="refund")

In [ ]:
sns.pointplot(delivery_data, y="sub_total",x="refund")

In [ ]:
print(f"p-value теста манауитни {mannwhitneyu(delivery_data.sub_total,delivery_data.refund).pvalue:.2f}")

In [ ]:
print(f"Средня цена товара  датасете: {1 / delivery_data.sub_total.mean():.2f}$")

In [ ]:
1 / delivery_data.groupby("refund")["sub_total"].describe().drop(["count","min","max","std"],axis=1)

Хоть и разницы между ценами оказались статистически значимы практической разница в данных очень мало (разница в )

## Скидка

In [ ]:
sns.barplot(delivery_data,y="refund",hue="discount")

In [ ]:
cross_tab = pd.crosstab(delivery_data.discount,delivery_data.refund)
cross_tab

In [ ]:
print(f"p-value Хи квадрат для переменных: {chi2_contingency(cross_tab).pvalue:.2f}")

## Чаевые

In [ ]:
sns.barplot(delivery_data,y="refund",hue="tip")

In [ ]:
sns.pointplot(delivery_data, y="refund", x="tip")

In [ ]:
cross_tab = pd.crosstab(delivery_data.tip, delivery_data.refund)
cross_tab

In [ ]:
print(f"p-value Хи-квадрат теста: {chi2_contingency(cross_tab).pvalue:.2f}")

## Время доставки

In [ ]:
sns.boxplot(delivery_data,y="delivery_time",hue="refund")

In [ ]:
sns.pointplot(delivery_data,x="refund",y="delivery_time")

In [ ]:
print(f"P-value теста Манауитни: {mannwhitneyu(delivery_data.delivery_time,delivery_data.refund).pvalue:.2f}")

# Создание модели

In [ ]:
from sklearn.linear_model  import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import RocCurveDisplay

In [ ]:
delivery_data.set_index("customer_id")
dataset = pd.get_dummies(delivery_data)

X = dataset.drop(["refund"],axis=1)
Y = dataset["refund"]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.25)

## Логистическая регрессия

### Без предварительной подготовки модели
Изучим поведение модели без предварительного изменения гиперпараметров

In [ ]:
parameters = [
    #Параметры для l1 регуляризации
    {"penalty":["l1"],
    "fit_intercept":[True,False],
    "solver":["liblinear"],
    "C": np.logspace(-4, 4, 10)
    },
    
    #Параметры для l2 регуляризации
    {"penalty":["l2"],
    "fit_intercept":[True,False],
    "solver":["liblinear", "lbfgs"],
    "C": np.logspace(-4, 4, 10)
    }
    ]

clf = LogisticRegression(random_state=42, max_iter=1000, class_weight="balanced")

search = GridSearchCV(clf,param_grid=parameters, cv = 5, scoring="f1", verbose=1, n_jobs=-1)
search.fit(X_train, y_train)

In [ ]:
clf = LogisticRegression(random_state=42, max_iter=1000, penalty="l1",class_weight="balanced",solver="liblinear")
clf.fit(X_train, y_train)

In [ ]:
RocCurveDisplay.from_estimator(clf,X_test,y_test, plot_chance_level=True)

### Cнижение параметров

In [ ]:
delivery_data.head()

In [ ]:
dataset = delivery_data
dataset = dataset.drop(["Date","time_ordered","time_delivered","delivery_area"],axis=1)
dataset = pd.get_dummies(dataset)

X = dataset.drop("refund", axis=1)
Y = dataset["refund"]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.25)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
RocCurveDisplay.from_estimator(clf,X_test,y_test, plot_chance_level=True)

##

## Дерево решений

In [ ]:
clf = DecisionTreeClassifier(random_state=42)
parameters = {"max_depth":range(1,10),"class_weight":["balanced",None]}
search = GridSearchCV(clf,param_grid=parameters)

In [ ]:
search.fit(X_train, y_train)

In [ ]:
plot_tree(search.best_estimator_,filled=True, feature_names=X_test.columns)

In [ ]:
RocCurveDisplay.from_estimator(search.best_estimator_,X_test,y_test, plot_chance_level=True)